# Calculate D Matrix without using @NLexpressions

In [ ]:
using NLOptControl, ForwardDiff


In [ ]:
tic()
D = zeros(Float64,Nc,Nc+1,Ni)
for int in 1:Ni
    for idx in 1:Nc
        for j in 1:Nc+1
            D[idx,j,int] = ForwardDiff.derivative(lagrange_basis_poly, t_data[idx,int], t_dataE[:,int], Nc, j)
        end
    end
end
toc()

# Using @NLexpressions to calculate DMatrix

In [1]:
using NLOptControl, JuMP, Ipopt
#FastGaussQuadrature

In [2]:
n = NLOpt();
const g = 1.62519; # m/s^2
function MoonLander{T<:Any}(n::NLOpt,x::Array{T,2},u::Array{T,2}) # dynamic constraint equations
  if n.integrationMethod==:tm
    L = size(x)[1];
  else
    L = size(x)[1]-1;
  end
  dx = Array(Any,L,n.numStates)
  dx[:,1] =  @NLexpression(mdl, [j=1:L], x[j,2] );
  dx[:,2] =  @NLexpression(mdl, [j=1:L], u[j,1] - g);
  return dx
end

n = define(n,stateEquations=MoonLander,numStates=2,numControls=1,X0=[10.,-2],XF=[0.,0.],XL=[-Inf,-Inf],XU=[Inf,Inf],CL=[0.],CU=[3.])
n = configure(n,Ni=1,Nck=[15];(:integrationMethod => :ps),(:integrationScheme => :lgrExplicit),(:finalTimeDV => false),(:tf => 4.0))
mdl = Model(solver = IpoptSolver()); 

# Fixed time function testing

In [3]:
n = LGR_matrices(n);
n.DMatrix[1]

15×16 Array{Any,2}:
 -28.25       38.3619     -15.6569     …   4.10797   -6.03972   3.75    
  -6.18903    -0.127062     9.0117        -1.67819    2.4662   -1.53107 
   1.38442    -4.93909     -0.132093       1.2934    -1.89857   1.17837 
  -0.553142    1.61374     -3.61382       -1.13583    1.66408  -1.03238 
   0.285687   -0.779796     1.35482        1.07125   -1.56493   0.970225
  -0.170867    0.45235     -0.715748   …  -1.06454    1.54858  -0.959171
   0.1124     -0.292672     0.442172       1.10708   -1.60061   0.990035
  -0.0790146   0.203702    -0.299729      -1.20405    1.72512  -1.06489 
   0.0582781  -0.149285     0.216069       1.37525   -1.94324   1.19588 
  -0.0445018   0.113508    -0.162514      -1.66693    2.30274  -1.41024 
   0.0347853  -0.0884627    0.125727   …   2.19154   -2.90766   1.76588 
  -0.0274974   0.0697861   -0.0986828     -3.28841    4.01135  -2.39712 
   0.0216041  -0.0547533    0.0771616      6.64046   -6.39119   3.67741 
  -0.0162873   0.0412422   -0.0

In [5]:
polyDiff(n.ts[1])[1:end-1,:]

15×16 Array{Float64,2}:
 -28.25       38.3619     -15.6569     …   4.10797   -6.03972   3.75    
  -6.18903    -0.127062     9.0117        -1.67819    2.4662   -1.53107 
   1.38442    -4.93909     -0.132093       1.2934    -1.89857   1.17837 
  -0.553142    1.61374     -3.61382       -1.13583    1.66408  -1.03238 
   0.285687   -0.779796     1.35482        1.07125   -1.56493   0.970225
  -0.170867    0.45235     -0.715748   …  -1.06454    1.54858  -0.959171
   0.1124     -0.292672     0.442172       1.10708   -1.60061   0.990035
  -0.0790146   0.203702    -0.299729      -1.20405    1.72512  -1.06489 
   0.0582781  -0.149285     0.216069       1.37525   -1.94324   1.19588 
  -0.0445018   0.113508    -0.162514      -1.66693    2.30274  -1.41024 
   0.0347853  -0.0884627    0.125727   …   2.19154   -2.90766   1.76588 
  -0.0274974   0.0697861   -0.0986828     -3.28841    4.01135  -2.39712 
   0.0216041  -0.0547533    0.0771616      6.64046   -6.39119   3.67741 
  -0.0162873   0.0412422   

# Variable time function testing

In [5]:
@variable(mdl, 0.01 <= tf <= Inf) 
n.tf = tf;
n = create_intervals(mdl,n,tf);
n = D_matrix(mdl,n);

In [11]:
setvalue(tf,3)
getvalue(tf)

3.0

In [12]:
getvalue(n.tf)

3.0

In [13]:
getvalue(n.ωₛ[1]);
getvalue(n.DMatrix[1])

15×16 Array{Float64,2}:
 -37.6667     51.1493     -20.8759     …   5.4773    -8.05296   5.0    
  -8.25204    -0.169415    12.0156        -2.23759    3.28827  -2.04143
   1.8459     -6.58545     -0.176124       1.72454   -2.53143   1.57116
  -0.737523    2.15166     -4.81842       -1.51444    2.21877  -1.3765 
   0.380916   -1.03973      1.80643        1.42833   -2.08657   1.29363
  -0.227823    0.603134    -0.95433    …  -1.41939    2.06477  -1.27889
   0.149867   -0.39023      0.589563       1.4761    -2.13415   1.32005
  -0.105353    0.271603    -0.399639      -1.60539    2.30016  -1.41986
   0.0777041  -0.199047     0.288092       1.83367   -2.59099   1.59451
  -0.0593358   0.151343    -0.216686      -2.22257    3.07032  -1.88032
   0.0463803  -0.11795      0.167636   …   2.92205   -3.87689   2.35451
  -0.0366632   0.0930481   -0.131577      -4.38454    5.34847  -3.19617
   0.0288054  -0.0730044    0.102882       8.85395   -8.52159   4.90321
  -0.0217164   0.0549895   -0.0773272   

In [14]:
poldif(getvalue(n.ts[1]))[1:end-1,:]

15×16 Array{Float64,2}:
 -37.6667     51.1493     -20.8759     …   5.4773    -8.05296   5.0    
  -8.25204    -0.169415    12.0156        -2.23759    3.28827  -2.04143
   1.8459     -6.58545     -0.176124       1.72454   -2.53143   1.57116
  -0.737523    2.15166     -4.81842       -1.51444    2.21877  -1.3765 
   0.380916   -1.03973      1.80643        1.42833   -2.08657   1.29363
  -0.227823    0.603134    -0.95433    …  -1.41939    2.06477  -1.27889
   0.149867   -0.39023      0.589563       1.4761    -2.13415   1.32005
  -0.105353    0.271603    -0.399639      -1.60539    2.30016  -1.41986
   0.0777041  -0.199047     0.288092       1.83367   -2.59099   1.59451
  -0.0593358   0.151343    -0.216686      -2.22257    3.07032  -1.88032
   0.0463803  -0.11795      0.167636   …   2.92205   -3.87689   2.35451
  -0.0366632   0.0930481   -0.131577      -4.38454    5.34847  -3.19617
   0.0288054  -0.0730044    0.102882       8.85395   -8.52159   4.90321
  -0.0217164   0.0549895   -0.0773272   

In [ ]:
julia> getvalue(n.DMatrix[1])
10×11 Array{Float64,2}:
 -464.59       815.336    -203.461     259.589    -83.0349    207.093    -56.6926    346.067   -53.9527  -352.112  -48.6547
  -93.2577      -4.77296   117.532    -122.661     36.7244    -88.886     23.9511   -144.902    22.4769   146.328   20.2084
   21.0219    -106.17       -5.21578   151.909    -35.3213     77.9651   -20.0967    118.705   -18.1723  -117.552  -16.2115
   -8.49468     35.0929    -48.1118     -6.03011   52.2645    -88.3321    20.5987   -115.906    17.3014   110.595   15.2126
    4.4471     -17.1958     18.3089    -85.5388    -7.4418    143.459    -25.5088    129.686   -18.465   -115.569  -15.825 
   -2.69499     10.113      -9.81979    35.1279   -34.8581     -9.95714   44.1086   -171.21     22.1169   133.098   18.0783
    1.78635     -6.59809     6.12877   -19.8344    15.0077   -106.8      -14.8543    311.664   -30.9859  -171.587  -22.937 
   -1.24421      4.55469    -4.13057    12.7344    -8.7058     47.3007   -35.5614    -26.1185   59.5713   260.432   33.5247
    0.868228    -3.16234     2.83034    -8.50828    5.54821   -27.3496    15.825    -266.64    -61.8372  -550.492  -62.2259
   -0.536311     1.94857    -1.7329      5.14771   -3.2867     15.5781    -8.29433   110.331   -52.1035  -319.163  211.243 

julia> poldif(getvalue(n.ts[1]),1)[1:end-1,:]
10×11 Array{Float64,2}:
 -464.59      633.882    -265.142    162.632    -119.828     99.6154   -91.2879   92.018   -104.473    149.173    -91.9981
 -101.565      -4.77296   153.163    -76.8466     52.9972   -42.7558    38.5667  -38.529     43.5238   -61.9921    38.2108
   22.8946    -82.5414     -5.21578   95.17      -50.9725    37.5026   -32.3601   31.5632   -35.1885    49.801    -30.6533
   -9.25141    27.2829    -62.6973    -6.03011    75.4232   -42.4894    33.1685  -30.819     33.5021   -46.854     28.7645
    4.84327   -13.3689     23.8594   -53.5897     -7.4418    69.0064   -41.0749   34.483    -35.7553    48.9609   -29.9225
   -2.93507     7.86231   -12.7967    22.0074    -50.304     -9.95714   71.0247  -45.524     42.8267   -56.3873    34.1831
    1.94549    -5.12968     7.98676  -12.4262     21.6577   -51.3727   -14.8543   82.8703   -60.0006    72.6933   -43.3701
   -1.35505     3.54104    -5.38279    7.97806   -12.5634    22.7525   -57.2618  -26.1185   115.353   -110.333     63.3898
    0.945573   -2.45856     3.68839   -5.33039     8.00666  -13.1557    25.4819  -70.8986   -61.8372   233.217   -117.659 
   -0.584088    1.51491    -2.25825    3.22501    -4.74306    7.49338  -13.3557   29.3367  -100.892   -319.163    399.426 


# OLD code

In [2]:
Ni=2;Nck=[10,5]; t0=0.0; tf = 4.0;
# initialize node data TODO  make different PS methods available
taus_and_weights = [gaussradau(Nck[int]) for int in 1:Ni];
τ = [taus_and_weights[int][1] for int in 1:Ni];
ω = [taus_and_weights[int][2] for int in 1:Ni];
ts, ωₛ = create_intervals(t0,tf,Ni,Nck,τ,ω);


In [3]:
function lepoly(n::Int64,x,derivative_option::Bool)
# lepoly  Legendre polynomial of degree n
    # y=lepoly(n,x) is the Legendre polynomial
    # The degree should be a nonnegative integer
    # The argument x should be on the closed interval [-1,1];
    # [dy,y]=lepoly(n,x) also returns the values of 1st-order
    #  derivative of the Legendre polynomial stored in dy
# Last modified on August 30, 2011
# Verified with the chart in http://keisan.casio.com/has10/SpecExec.cgi
  if !derivative_option
       if n==0 y=ones(size(x));  return y end
       if n==1 y=x; return y end
       polylst=ones(size(x)); poly=x;   # L_0(x)=1, L_1(x)=x
       polyn=0;
       for k=2:n                      # Three-term recurrence relation:
  	     polyn=((2*k-1)*x.*poly-(k-1)*polylst)/k; # kL_k(x)=(2k-1)xL_{k-1}(x)-(k-1)L_{k-2}(x)
         polylst=poly; poly=polyn;
       end
       y=polyn;
  else
     if n==0 y=ones(size(x)); dy=zeros(size(x)); return dy,y end
     if n==1 y=x; dy=ones(size(x)); return dy, y end

     # TODO MAKE SURE THIS IS CALCULATING SIZE correctly!! i.e. N=4 and size(N+1,N+1) = 1
      polylst=ones(size(x)); pderlst=zeros(size(x));poly=x; pder=ones(size(x));
       # L_0=1, L_0'=0, L_1=x, L_1'=1
      polyn=0;
      pdern=0;
      for k=2:n                          # Three-term recurrence relation:
        polyn=((2*k-1)*x.*poly-(k-1)*polylst)/k; # kL_k(x)=(2k-1)xL_{k-1}(x)-(k-1)L_{k-2}(x)
        pdern=pderlst+(2*k-1)*poly; # L_k'(x)=L_{k-2}'(x)+(2k-1)L_{k-1}(x)
     	  polylst=poly; poly=polyn;
    	  pderlst=pder; pder=pdern;
      end
      y=polyn; dy=pdern;
      return dy, y
  end

end


# the following code use

lepoly (generic function with 1 method)

In [4]:
#--------------------------------------------------------------------------
# Author: Huckleberry Febbo, Graduate Student, University of Michigan
# Date: 12/23/2015
# Last modifed on December 23, 2016 by Huckleberry Febbo
#--------------------------------------------------------------------------
# From a modified version of what is described in the paper (*) titled:
# "Legendre–Gauss–Lobatto Pseudo–spectral Method for One–Dimensional Advection–Diffusion Equation"
function MYD(τ::Vector{Float64},N::Int64)
  D = zeros(N+1,N+1)
  for i in 1:N+1
    for j in 1:N+1
      if i!=j
        D[i,j] = lepoly(N,τ[i],false)[1]/(lepoly(N,τ[j],false)[1]*(τ[i]-τ[j]));
      elseif (i==j && j==1)
        D[i,j] = -N*(N+1)/4;
      elseif (i==j && j==(N+1)) #TODO added the N+1 --> check this (what I modified from the paper(*))
        D[i,j] = N*(N+1)/4;
      else
        D[i,j] = 0;
      end
    end
  end
  return D
end


THIS IS THE WRONG D! -- > not the same;/

LoadError: syntax: extra token "IS" after end of expression

In [5]:
MYD(ts[1],10)

11×11 Array{Float64,2}:
    -27.5           -19.0536     …   1.53533e-6     1.28093e-6 
      9.98069         0.0            1.15365e-6     9.6196e-7  
     -3.45714        -6.89338       -1.42411e-6    -1.18581e-6 
      1.04415         1.70303        1.00144e-6     8.31702e-7 
     -1.56271        -2.38567       -2.99165e-6    -2.47339e-6 
    -44.8299        -66.4165     …  -0.000163099   -0.000133757
  -2754.3         -4016.52          -0.0194894     -0.01573    
 -23711.8        -34270.4           -0.365626      -0.284175   
 -85997.7            -1.23666e5     -4.01674       -2.7414     
     -1.67628e5      -2.40455e5      0.0          -29.3671     
     -1.95171e5      -2.79809e5  …  40.9831        27.5        

In [6]:
using JuMP
mdl = Model()
@variable(mdl, tf_var)
ts_JuMP, ωₛ_JuMP = create_intervals_JuMP(mdl,tf_var,Nck,Ni,τ,ω)
DJ=poldif_JuMP(mdl,ts_JuMP,Ni,Nck);
setvalue(tf_var, tf)

In [7]:
getvalue(ts_JuMP[2])

6-element Array{Any,1}:
 2.0    
 2.27952
 2.83282
 3.44631
 3.88579
 4.0    

In [8]:
tm

3-element Array{Float64,1}:
 0.0
 2.0
 4.0

In [9]:
tm = @NLexpression(mdl, [idx=1:Ni+1], (idx-1)*tf_var/Ni);
getvalue(tm)

3-element Array{Float64,1}:
 0.0
 2.0
 4.0

In [10]:
ts[2]

6-element Array{Float64,1}:
 2.0    
 2.27952
 2.83282
 3.44631
 3.88579
 4.0    

In [14]:
getvalue(DJ[1])

11×11 Array{Float64,2}:
 -25.25       34.4508     -14.4102    …  -5.678      8.10738  -5.0    
  -5.51997    -0.259405     8.32426       2.36547   -3.3692    2.07672
   1.2443     -4.48604     -0.283472     -1.91246    2.70663  -1.66597
  -0.502804    1.4828      -3.40753       1.8208    -2.54646   1.56332
   0.263226   -0.726584     1.29673      -1.94326    2.66097  -1.62626
  -0.159518    0.427308    -0.695489  …   2.32759   -3.06459   1.85781
   0.105735   -0.278793     0.434072     -3.26097    3.95081  -2.35712
  -0.0736453   0.192452    -0.292549      6.2693    -5.99646   3.44517
   0.0513909  -0.13362      0.20046      -3.36078   12.6751   -6.39464
  -0.0317445   0.0823339   -0.122733     -5.48338  -17.3462   21.7084 
   0.05       -0.129611     0.192936  …   7.06521  -55.442    50.0    

In [15]:
poldif(ts[1],1)

11×11 Array{Float64,2}:
 -25.25       34.4508     -14.4102    …  -5.678      8.10738  -5.0    
  -5.51997    -0.259405     8.32426       2.36547   -3.3692    2.07672
   1.2443     -4.48604     -0.283472     -1.91246    2.70663  -1.66597
  -0.502804    1.4828      -3.40753       1.8208    -2.54646   1.56332
   0.263226   -0.726584     1.29673      -1.94326    2.66097  -1.62626
  -0.159518    0.427308    -0.695489  …   2.32759   -3.06459   1.85781
   0.105735   -0.278793     0.434072     -3.26097    3.95081  -2.35712
  -0.0736453   0.192452    -0.292549      6.2693    -5.99646   3.44517
   0.0513909  -0.13362      0.20046      -3.36078   12.6751   -6.39464
  -0.0317445   0.0823339   -0.122733     -5.48338  -17.3462   21.7084 
   0.05       -0.129611     0.192936  …   7.06521  -55.442    50.0    

In [13]:
GOOD!!

LoadError: syntax: extra token "first" after end of expression

In [20]:
getvalue(ωₛ_JuMP[2])

5-element Array{Any,1}:
 0.08    
 0.446208
 0.623653
 0.562712
 0.287427

In [19]:
ωₛ[2]

5-element Array{Float64,1}:
 0.08    
 0.446208
 0.623653
 0.562712
 0.287427

In [ ]:
GOOD!!!